# SQL Python

Tenemos 3 CSV, debemos crear una base de datos en Dbeaver y usando python ingresar los datos y después realizarnos 5 preguntas al respecto que podamos sacar con consultas que lleven JOIN, GROUP BY o similar

También podemos practicar subsconsultas

In [48]:
# importamos las librerías con las que vamos a trabajar

# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd

from geopy.geocoders import Nominatim
import dotenv    
import os

from datetime import datetime

Primero voy a cargar los CSV con objetivo de ver que tablas pueda sacar

In [2]:
df_municipios = pd.read_csv("datos/output/fixed_municipios.csv",index_col="Unnamed: 0")
df_categorias = pd.read_csv("datos/output/categorias.csv",index_col="Unnamed: 0")
df_estado_cielo = pd.read_csv("datos/output/estado_cielo.csv",index_col="Unnamed: 0")
df_fechas = pd.read_csv("datos/output/fechas.csv",index_col="Unnamed: 0")
df_tiempo = pd.read_csv("datos/output/tabla_aemet.csv",index_col="Unnamed: 0")
df_lugares = pd.read_csv("datos/output/lugares.csv",index_col="Unnamed: 0")

# Municipios
- Municipios
    - id_municipio PK
    - nombre
    - latitud
    - longitud


In [3]:
df_municipios.sample()

,municipio sucio,latitud,longitud,nombre municipio
10,ambite,40.328972,-3.181211,Ambite


In [4]:
df_municipios = df_municipios.drop(columns=["municipio sucio"])

In [5]:
df_municipios = df_municipios[["nombre municipio","latitud","longitud"]]
df_municipios.sample()

,nombre municipio,latitud,longitud
119,Robregordo,41.106452,-3.593747


In [6]:
df_municipios.to_csv("datos/output/municipios_table.csv")

In [7]:
df_municipios = pd.read_csv("datos/output/municipios_table.csv",index_col="Unnamed: 0")

In [8]:
df_municipios.sample()

,nombre municipio,latitud,longitud
39,Chinchón,40.140829,-3.423147


In [26]:
key = os.getenv("sql")
try:
    conexion = psycopg2.connect(
        database = "laboratorio3",
        user = "postgres",
        password = key,#Esto en un .env
        host = "localhost",
        port = "5432" )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("La contraseña es errónea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión")
    else:
        print(f"Ocurrió el error {e}")


In [27]:
cursor = conexion.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS "municipios"(
                    id_municipio SERIAL PRIMARY KEY,
                    nombre VARCHAR(100),
                    latitud NUMERIC(8,6),
                    longitud NUMERIC(8,6));''')
conexion.commit()

In [28]:
tuplita_municipios = [tuple(v) for v in df_municipios.values]
tuplita_municipios[:3]

[('La Acebeda', 41.0869583, -3.624399),
 ('Ajalvir', 40.5342302, -3.4807818),
 ('Alameda del Valle', 40.9177178, -3.8438216)]

### Insertar Valores
Lo comento por mi seguridad
```python
    query_insertar = """ INSERT INTO municipios(nombre,latitud,     longitud) VALUES (%s,%s,%s)"""
    cursor.executemany(query_insertar,tuplita_municipios)
    conexion.commit()
```

In [19]:
cursor.close()
conexion.close()

# Categorías
- Categorías
    - id_categoria PK
    - tipo


In [13]:
df_categorias.sample()

,categoria
6,Fountain


In [20]:
key = os.getenv("sql")
try:
    conexion = psycopg2.connect(
        database = "laboratorio3",
        user = "postgres",
        password = key,#Esto en un .env
        host = "localhost",
        port = "5432" )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("La contraseña es errónea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión")
    else:
        print(f"Ocurrió el error {e}")

In [21]:
cursor = conexion.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS "categorias"(
                    id_categoria SERIAL PRIMARY KEY,
                    nombre VARCHAR(100));''')
conexion.commit()

In [22]:
tuplita_categorias = [tuple(v) for v in df_categorias.values]
tuplita_categorias[:3]

[('Park',), ('Monument',), ('TV Station',)]

```python
query_insertar = """ INSERT INTO categorias(nombre) VALUES (%s)"""
cursor.executemany(query_insertar,tuplita_categorias)
conexion.commit()
```

In [30]:
cursor.close()
conexion.close()

# Estado Cielo
- Estado cielo
    - id_estado PK
    - estado


In [14]:
df_estado_cielo.sample()

,estado_cielo
9,Cubierto con lluvia


In [33]:
key = os.getenv("sql")
try:
    conexion = psycopg2.connect(
        database = "laboratorio3",
        user = "postgres",
        password = key,#Esto en un .env
        host = "localhost",
        port = "5432" )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("La contraseña es errónea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión")
    else:
        print(f"Ocurrió el error {e}")

In [34]:
cursor = conexion.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS "estado_cielo"(
                    id_estado_cielo SERIAL PRIMARY KEY,
                    nombre VARCHAR(100));''')
conexion.commit()

In [35]:
tuplita_cielo = [tuple(v) for v in df_estado_cielo.values]
tuplita_cielo[:3]

[('Nubes altas',), ('Muy nuboso',), ('Cubierto con tormenta y lluvia escasa',)]

```python
query_insertar = """ INSERT INTO estado_cielo(nombre) VALUES (%s)"""
cursor.executemany(query_insertar,tuplita_cielo)
conexion.commit()

In [37]:
cursor.close()
conexion.close()

# Fecha
- Fecha
    - id_fecha PK
    - Fecha


In [15]:
df_fechas["fecha"] = pd.to_datetime(df_fechas["fecha"])
df_fechas.sample()

,fecha
31,2024-08-30 19:00:00


In [66]:
key = os.getenv("sql")
try:
    conexion = psycopg2.connect(
        database = "laboratorio3",
        user = "postgres",
        password = key,#Esto en un .env
        host = "localhost",
        port = "5432" )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("La contraseña es errónea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión")
    else:
        print(f"Ocurrió el error {e}")

In [67]:
cursor = conexion.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS "fechas"(
                    id_fecha SERIAL PRIMARY KEY,
                    fecha TIMESTAMP);''')
conexion.commit()

In [68]:
from datetime import datetime
tuplita_fechas = [tuple(v) for v in df_fechas.values]   


In [69]:
fechas_convertidas = [t[0].astype('M8[ms]').astype(datetime) for t in tuplita_fechas]

In [70]:
tuplita_fechas = [(fecha,) for fecha in fechas_convertidas]

In [71]:
tuplita_fechas[:3]

[(datetime.datetime(2024, 8, 29, 12, 0),),
 (datetime.datetime(2024, 8, 29, 13, 0),),
 (datetime.datetime(2024, 8, 29, 14, 0),)]

```py
query_insertar = """ INSERT INTO fechas(fecha) VALUES (%s)"""
cursor.executemany(query_insertar,tuplita_fechas)
conexion.commit()

In [74]:
cursor.close()
conexion.close()

# Tiempo

- Tiempo
    - id_tiempo PK
    - id_estado_cielo (FK estado cielo)
    - id_municipio (FK municipio)
    - id_fecha (FK fecha)
    - temperatura
    - sensacion_temp
    - racha_max
    - precipitación
    - nieve
    - humedad
    - probabilidad precipitación
    - probabilidad nieve
    - probabilidad tormenta
    - Dirección viento

In [16]:
df_tiempo.sample()

,indice_fecha,indice_estado_cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._de_nieve_(%),prob._de_tormenta_(%),dirección_viento,velocidad_del_viento,indice_municipio
7724,16,6,17,17,30.0,0.0,0,100,0,0,SE,21,173


In [86]:
key = os.getenv("sql")
try:
    conexion = psycopg2.connect(
        database = "laboratorio3",
        user = "postgres",
        password = key,#Esto en un .env
        host = "localhost",
        port = "5432" )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("La contraseña es errónea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión")
    else:
        print(f"Ocurrió el error {e}")

In [87]:
cursor = conexion.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS "tiempo"(
                    id_tiempo SERIAL PRIMARY KEY,
                    id_fecha INT NOT NULL,
                    id_estado_cielo INT NOT NULL,
                    temperatura INT NOT NULL,
                    sensacion_termica INT NOT NULL,
                    racha_max NUMERIC NOT NULL,
                    precipitacion NUMERIC NOT NULL,
                    nieve INT NOT NULL,
                    humedad_relativa INT NOT NULL,
                    probabilidad_de_nieve INT NOT NULL,
                    probabilidiento VARCHAR(3) NOT NULL,
                    velocidad_del_viento INT NOT NULL,
                    id_municipio INT NOT NULL,
                    FOREIGN KEY (id_estado_cielo) REFERENCES estado_cielo (id_estado_cielo) ON UPDATE CASCADE ON DELETE CASCADE,
                    FOREIGN KEY (id_municipio) REFERENCES municipios (id_municipio) ON UPDATE CASCADE ON DELETE CASCADE,
                    FOREIGN KEY (id_fecha) REFERENCES fechas (id_fecha)ad_de_tormenta INT NOT NULL,
                    direccion_v ON UPDATE CASCADE ON DELETE CASCADE);''')
conexion.commit()

In [88]:
tuplita_tiempo = [tuple(v) for v in df_tiempo.values]

In [89]:
tuplita_tiempo[:3]

[(1, 1, 21, 21, 25.0, 0.0, 0, 69, 0, 75, 'S', 7, 1),
 (2, 1, 24, 24, 29.0, 0.0, 0, 57, 0, 75, 'SE', 12, 1),
 (3, 2, 23, 23, 33.0, 0.0, 0, 59, 0, 80, 'S', 14, 1)]

```py
query_insertar = """ INSERT INTO tiempo(id_fecha,id_estado_cielo,temperatura,sensacion_termica,racha_max,precipitacion,nieve,humedad_relativa,probabilidad_de_nieve,probabilidad_de_tormenta,direccion_viento,velocidad_del_viento,id_municipio) 
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
cursor.executemany(query_insertar,tuplita_tiempo)
conexion.commit()

In [92]:
cursor.close()
conexion.close()

# Lugares
- Lugares
    - id_lugar PK
    - id_categoría (FK categorías)
    - id_municipio (FK municipio)
    - id_tiempo (FK tiempo)
    - nombre
    - dirección
    - distancia (al centro del municipio)

In [17]:
df_lugares.sample()

,indice_municipio,indice_categoria,name,address,distance
342,67,1,Parque del Ayuntamiento Guadarrama,28440 Guadarrama Comunidad de Madrid,25.0


In [100]:
df_lugares["address"] = df_lugares["address"].fillna("N/A")

In [101]:
key = os.getenv("sql")
try:
    conexion = psycopg2.connect(
        database = "laboratorio3",
        user = "postgres",
        password = key,#Esto en un .env
        host = "localhost",
        port = "5432" )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("La contraseña es errónea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión")
    else:
        print(f"Ocurrió el error {e}")

In [102]:
cursor = conexion.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS "lugares"(
                    id_lugar SERIAL PRIMARY KEY,
                    id_municipio INT NOT NULL,
                    id_categoria INT NOT NULL,
                    nombre VARCHAR(100) NOT NULL,
                    direccion VARCHAR(1000) NOT NULL,
                    distancia_al_centro NUMERIC NOT NULL,
                    FOREIGN KEY (id_municipio) REFERENCES municipios (id_municipio) 
                    ON UPDATE CASCADE ON DELETE CASCADE,
                    FOREIGN KEY (id_categoria) REFERENCES categorias (id_categoria) 
                    ON UPDATE CASCADE ON DELETE CASCADE);''')
conexion.commit()

In [103]:
tuplita_lugares = [tuple(v) for v in df_lugares.values]
tuplita_lugares[:3]

[(2,
  1,
  'Diverjungla',
  'Calle Segovia, 20 (Pol. Ind. Donada), 28864 Ajalvir Comunidad de Madrid',
  563.0),
 (2, 1, 'Parque Infantil', 'N/A', 1724.0),
 (2, 1, 'Chiquitin', 'Ajalvir Madrid', 333.0)]

```py
query_insertar = """ INSERT INTO lugares(id_municipio,id_categoria,nombre,direccion,distancia_al_centro) 
VALUES (%s,%s,%s,%s,%s)"""
cursor.executemany(query_insertar,tuplita_lugares)
conexion.commit()

In [106]:
cursor.close()
conexion.close()